In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"
!pip install swig
!pip install gymnasium[box2d]

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Video recording/playing functions
import os
import base64
from pathlib import Path
from IPython import display as ipythondisplay
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

def show_videos(video_path="", prefix=""):
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

In [ ]:
# Set environment name you like. See https://gymnasium.farama.org/environments/classic_control/

env_name = "CartPole-v1"
# env_name = "LunarLander-v2"
# env_name = "FrozenLake-v1"

env = gym.make(env_name)

model = PPO(MlpPolicy, env, verbose=1)

In [ ]:
# Show video and calculate mean reward before learning

record_video(env_name, model, video_length=500, prefix=f"ppo-{env_name}")
show_videos("videos", prefix=f"ppo-{env_name}")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, warn=False)
print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# Learning
model.learn(total_timesteps=20000)

In [ ]:
# Show video and calculate mean reward after learning

record_video(env_name, model, video_length=500, prefix=f"ppo-{env_name}-trained")
show_videos("videos", prefix=f"ppo-{env_name}-trained")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")